<a href="https://colab.research.google.com/github/SarahBarbosa/AluraDataChallenge_2/blob/main/S02_Feature_Engineering_e_Modelo_de_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://imgur.com/QnT31vZ.png)

---

# Sobre o desafio

Na semana 02, vamos focar na construção e otimização de modelos de machine learning (ML) para lidar com a Taxa de Churn. Agora que terminamos a limpeza e análise inicial dos dados na semana 01, é hora de usar algoritmos de ML para transformar nossos insights em ações eficazes.

## Configurando o ambiente

In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Github/AluraDataChallenge_2')

## Carregando os dados tratados na Semana 01

In [2]:
import pandas as pd

DIRETORIO_BASE = '/content/drive/MyDrive/Github/AluraDataChallenge_2/'
DIRETORIO_DADOS = DIRETORIO_BASE + 'Dados/'

dados = pd.read_csv(DIRETORIO_DADOS + 'dados_novexus_tratado.csv')

**Observação:** Antes de avançar com nossas análises e modelagem, vamos modificar o nome das colunas para simplificar nosso trabalho (poderíamos ter feito isso na semana 01, mas ainda dá tempo):

In [3]:
import funcoes_auxiliares as func_aux

dados = dados.rename(columns = func_aux.renomeia_coluna)

In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Churn                                  7043 non-null   int64  
 1   Gender                                 7043 non-null   int64  
 2   Seniorcitizen                          7043 non-null   int64  
 3   Partner                                7043 non-null   int64  
 4   Dependents                             7043 non-null   int64  
 5   Tenure                                 7043 non-null   int64  
 6   Multiplelines                          7043 non-null   int64  
 7   Onlinesecurity                         7043 non-null   int64  
 8   Onlinebackup                           7043 non-null   int64  
 9   Deviceprotection                       7043 non-null   int64  
 10  Techsupport                            7043 non-null   int64  
 11  Stre

<div style="background-color: #171821; color: white; padding: 10px; text-align: center; font-size: 20px;">
    <h1>Feature Engineering</h1>
</div>

Após as análises detalhadas feitas na semana 01, ficou claro que é possível criar novas variáveis e segmentação dos dados para tornar nossas features mais úteis para a construção dos modelos ML.

## Criando segmentos anuais para Tenure e segmentos para Charges_monthly e Charges_total

Sabemos que a variável `Tenure` representa o período de contrato em meses de cada cliente. Para uma melhor análise, decidimos categorizar essa feature em intervalos anuais. Vale mencionar que o intervalo mínimo é de 1 mês, enquanto o máximo é de 72 meses (6 anos).

> Categorias: `['0-1 Ano', '1-2 Anos', '2-3 Anos', '3-4 Anos', '4-5 Anos', '5-6 Anos']`

Além disso, podemos fazer uma segmentação nos gastos mensais (`Charges_Monthly`) e nas gastos totais (`Charges_Total`). Essa abordagem nos permite identificar grupos de clientes com diferentes níveis de despesas, o que é mais informativo do que considerar apenas valores contínuos.

> Categorias: `['Baixo', 'Moderado', 'Alto', 'Muito Alto']`

In [5]:
import funcoes_feat_eng as ffe

# Cria três colunas categórias: Tenure_anual, Charges_monthly_categories, Charges_total_categories
ffe.criar_segmentos(dados)

In [6]:
dados.iloc[:, -3:].head()

,Tenure_anual,Charges_monthly_categories,Charges_total_categories
0,0-1 Ano,Moderado,Baixo
1,0-1 Ano,Moderado,Baixo
2,0-1 Ano,Alto,Baixo
3,1-2 Anos,Muito Alto,Baixo
4,0-1 Ano,Alto,Baixo


## Identificando Pessoas que não recebem nenhum suporte, backup ou proteção

Podemos também identificar quais são os clientes que não recebem nenhum dos serviços de suporte (`Onlinesecurity`), backup (`Onlinebackup`) ou proteção (`Deviceprotection`), pois vimos na análise da Semana 01 que uma considerável quantidade de clientes que não possuiam essses serviços optaram deixar a empresa Novexus.

In [7]:
ffe.criar_indicador_internet_suporte(dados)

In [8]:
dados.iloc[:, -1:].value_counts(normalize = True)

No_internet_support_service
1                              0.901604
0                              0.098396
dtype: float64

## Número total de serviços de internet recebidos pelo cliente

Calculamos o número total de serviços recebidos por cada cliente com base em várias condições relacionadas aos serviços de internet. Isso nos dá uma visão geral do alcance dos serviços utilizados pelos clientes.

In [9]:
ffe.criar_indicador_todos_servicos_internet(dados)

In [10]:
dados.iloc[:, -1:].value_counts(normalize = True)

Num_internet_services
0                        0.400540
3                        0.137157
2                        0.126225
1                        0.120971
4                        0.107341
5                        0.070992
6                        0.036774
dtype: float64

## Calculando o gasto médio mensal e a média e o custo por serviço

- O gasto médio mensal de cada cliente pode ser obtido dividindo o valor total dos gastos (`Charges_total`) pelo tempo de permanência do cliente (`Tenure`).

- Por fim, para determinar o custo médio por serviço, dividimos o valor mensal dos gastos pelo número total de serviços mais um (adicionando "+1" para evitar divisões por zero quando não há nenhum serviço contratado).

In [11]:
ffe.calcular_estatisticas_gastos(dados)

In [12]:
dados.iloc[:, -2:].head()

,Avg_charges_monthly,Avg_charges_services
0,65.922222,148.325000
1,60.266667,271.200000
2,70.212500,140.425000
3,95.219231,247.570000
4,89.133333,89.133333


## Encoding adicional e Feature Scaling

Como inserimos novas colunas categórias, precisamos transforma-las em valores binários (agora vamos usar o `pd.get_dummies()` que evitamos usar na semana 01):

In [13]:
dados = pd.get_dummies(dados).copy()

Agora, precisamos ajustar os dados que estão em uma escala consideravelmente maior do que 0 e 1 para que todos estejam na mesma escala. Isso nos permitirá padronizar essas features, removendo a média e dimensionando para uma variação unitária.

In [14]:
from sklearn.preprocessing import StandardScaler

colunas_numericas = [col for col in dados.columns if dados[col].nunique() > 2]
scaler = StandardScaler()
dados[colunas_numericas] = scaler.fit_transform(dados[colunas_numericas])

In [15]:
dados[colunas_numericas].head()

,Tenure,Charges_monthly,Charges_total,Num_internet_services,Avg_charges_monthly,Avg_charges_services
0,-0.954310,0.027862,-0.745055,0.649373,0.038404,-0.911275
1,-0.954310,-0.161583,-0.767522,-0.423849,-0.148943,-0.740999
2,-1.158162,0.303720,-0.882969,-0.423849,0.180524,-0.922223
3,-0.791228,1.104706,-0.460553,1.185984,1.008900,-0.773744
4,-1.198932,0.636080,-0.888905,0.112762,0.807298,-0.993302


<div style="background-color: #171821; color: white; padding: 10px; text-align: center; font-size: 20px;">
    <h1>Lidando com o desbalanceamento de dados da target</h1>
</div>

Durante a primeira semana do nosso projeto, uma observação veio à tona: nossos dados da target 'Churn' apresentam um desbalanceamento, conforme evidenciado abaixo:

In [16]:
round(dados['Churn'].value_counts(normalize = True) * 100, 2)

0    73.46
1    26.54
Name: Churn, dtype: float64

Este desbalanceamento pode trazer desafios na construção de modelos ML. Na verdade, temos duas abordagens distintas a serem consideradas para lidar com essa situação:

## Abordagem 1: Ignorar o desbalanceamento

> A primeira abordagem consiste em ignorar completamente o desbalanceamento e construir nosso modelo sem fazer ajustes específicos para corrigir. Se desenvolvermos um modelo sem considerar essa desproporcionalidade nos dados, o modelo será vítima do Paradoxo da Acurácia, em que os parâmetros do algoritmo não diferenciarão a classe minoritária das demais categorias, acreditando que estão agregando resultado devido à aparente alta acurácia. No entanto, existem diversos algoritmos de classificação, como Gradient Boosting, por exemplo, que demonstram um desempenho melhor lidando com dados desbalanceados do que modelos como o KNN e SVM.


In [17]:
X_ignore = dados.drop('Churn', axis = 1)   # Conjunto de features
y_ignore = dados['Churn']                  # Conjunto da target Churn

## Abordagem 2: Oversamplig

> A segunda abordagem é utilizar a técnica [SMOTE](https://arxiv.org/pdf/1106.1813.pdf) (Synthetic Minority Over-sampling Technique). Essa técnica envolve a criação de informações sintéticas com base nas observações já existentes na classe minoritária. Esses dados "sintéticos" são gerados de forma a serem semelhantes aos dados reais, embora não sejam idênticos. Essa abordagem visa equilibrar a representação das classes, tornando o modelo mais capaz de identificar corretamente os casos da classe minoritária.

In [18]:
from imblearn.over_sampling import SMOTE

X_oversample, y_oversample = SMOTE(random_state= 42).fit_resample(X_ignore, y_ignore)

A partir deste ponto, nosso trabalho se ramificará nessas duas abordagens distintas. Ao final, identificaremos qual delas é a mais eficaz, juntamente com a escolha do modelo de classificação adequado. Também discutiremos as medidas que a empresa Novexus pode adotar para reduzir a taxa de Churn.